# Training Code

In [1]:
%pwd

'/media/manralai/Drive D/ImageClassifier-Pipeline/research_env'

In [2]:
# to root
%cd ..

/media/manralai/Drive D/ImageClassifier-Pipeline


In [18]:
import warnings
warnings.filterwarnings("ignore")

### Creating Entity Realted with Trainig Configuration

In [19]:
from pathlib import Path
from dataclasses import dataclass



@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list


@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

### Configuration Manager Class

In [24]:
import os
import tensorflow as tf
from imageClassifier.constants import *
from imageClassifier.utils import read_yaml, create_directories



class ConfigurationManager:
    def __init__(self,config_filepath = CONFIG_FILE_PATH,params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_prepare_callback_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([Path(model_ckpt_dir),Path(config.tensorboard_root_log_dir)])

        prepare_callback_config = PrepareCallbacksConfig(root_dir=Path(config.root_dir),
                                                         tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
                                                         checkpoint_model_filepath=Path(config.checkpoint_model_filepath))
        return prepare_callback_config
    

    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir,"classifier_data_v01") #"updated folder from PetImages"
        create_directories([Path(training.root_dir)])

        training_config = TrainingConfig(root_dir=Path(training.root_dir),
                                         trained_model_path=Path(training.trained_model_path),
                                         updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
                                         training_data=Path(training_data),
                                         params_epochs=params.EPOCHS,
                                         params_batch_size=params.BATCH_SIZE,
                                         params_is_augmentation=params.AUGMENTATION,
                                         params_image_size=params.IMAGE_SIZE)
        return training_config

### Preparing Callbacks

In [25]:
import time



class PrepareCallback:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config

    
    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(self.config.tensorboard_root_log_dir,f"tb_logs_at_{timestamp}",)
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    

    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(filepath=str(self.config.checkpoint_model_filepath),save_best_only=True)


    def get_tb_ckpt_callbacks(self):
        return [self._create_tb_callbacks,self._create_ckpt_callbacks]



### Training Components

In [26]:
import os
import time
import tensorflow as tf
from zipfile import ZipFile
import urllib.request as request



class Training:
    #initialization : Training class is initialized with a TrainingConfig object, 
    #                   which likely contains configuration parameters for training process.
    def __init__(self, config:TrainingConfig):
        self.config = config
    
    #loading base model : method loads a pre-existing base model specified by updated_base_model_path from configuration.
    def get_base_model(self):
        self.model = tf.keras.models.load_model(self.config.updated_base_model_path)
    
    #generatorSetup : seting up image data generators for both training and validation
    #                   uses TensorFlow's ImageDataGenerator to perform data augmentation 
    #                   during training if specified by params_is_augmentation in the configuration    
    def train_valid_generator(self):
        datagenerator_kwargs = dict(rescale = 1./255,validation_split=0.20)
        dataflow_kwargs = dict(target_size=self.config.params_image_size[:-1],
                               batch_size=self.config.params_batch_size,
                               interpolation="bilinear")
        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(**datagenerator_kwargs)

        self.valid_generator = valid_datagenerator.flow_from_directory(directory=self.config.training_data,
                                                                       subset="validation",
                                                                       shuffle=False,
                                                                       **dataflow_kwargs)
        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(rotation_range=40,
                                                                                  horizontal_flip=True,
                                                                                  width_shift_range=0.2,
                                                                                  height_shift_range=0.2,
                                                                                  shear_range=0.2,
                                                                                  zoom_range=0.2,
                                                                                  **datagenerator_kwargs)
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(directory=self.config.training_data,
                                                                       subset="training",
                                                                       shuffle=True,
                                                                       **dataflow_kwargs)
    #Saving Trained Model : static method is used to save trained model to a specified path
    @staticmethod
    def save_model(path:Path, model:tf.keras.Model):
        model.save(path)


    #Training Process
    def train(self, callback_list:list):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(self.train_generator,
                       epochs=self.config.params_epochs,
                       steps_per_epoch=self.steps_per_epoch,
                       validation_steps=self.validation_steps,
                       validation_data=self.valid_generator,
                       callbacks=callback_list)
        self.save_model(path=self.config.trained_model_path,model=self.model)

In [29]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callback_config()
    prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()

    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train(callback_list=callback_list)
except Exception as e:
    raise e

[2024-01-06 15:10:04,116: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-01-06 15:10:04,126: INFO: common: yaml file: params.yaml loaded successfully]
[2024-01-06 15:10:04,130: INFO: common: Created directory at: artifacts]
[2024-01-06 15:10:04,133: INFO: common: Created directory at: artifacts/prepare_callbacks/checkpoint_dir]
[2024-01-06 15:10:04,136: INFO: common: Created directory at: artifacts/prepare_callbacks/tensorboard_log_dir]
[2024-01-06 15:10:04,140: INFO: common: Created directory at: artifacts/training]


Found 200 images belonging to 2 classes.
Found 800 images belonging to 2 classes.
Epoch 1/20
30/30 [==============================] - 90s 3s/step - loss: 13.1497 - accuracy: 0.5530 - val_loss: 20.2577 - val_accuracy: 0.5000
Epoch 2/20
30/30 [==============================] - 94s 3s/step - loss: 14.7692 - accuracy: 0.5284 - val_loss: 8.2784 - val_accuracy: 0.5055
Epoch 3/20
30/30 [==============================] - 92s 3s/step - loss: 11.6622 - accuracy: 0.5698 - val_loss: 8.7291 - val_accuracy: 0.5330
Epoch 4/20
30/30 [==============================] - 93s 3s/step - loss: 11.1315 - accuracy: 0.5685 - val_loss: 13.8611 - val_accuracy: 0.5000
Epoch 5/20
30/30 [==============================] - 101s 3s/step - loss: 10.0728 - accuracy: 0.5801 - val_loss: 5.6549 - val_accuracy: 0.6593
Epoch 6/20
30/30 [==============================] - 97s 3s/step - loss: 11.5021 - accuracy: 0.5401 - val_loss: 10.2535 - val_accuracy: 0.5495
Epoch 7/20
30/30 [==============================] - 100s 3s/step - l